# Ejemplo Problema de dieta

Tomado de 

https://www.gestiondeoperaciones.net/programacion_lineal/problema-de-la-dieta-en-programacion-lineal-resuelto-con-solver-de-excel/

In [1]:
import pandas as pd
from pyomo.environ import *


In [14]:
InfoAlimentos = ['TamPorcion', 'Energia', 'Proteinas', 'Calcio',
       'PrecioPorcion', 'LimPorcionesDia']
df = pd.read_excel("Alimentos.xlsx",index_col="Alimento",names=InfoAlimentos)

In [15]:
df.columns

Index(['TamPorcion', 'Energia', 'Proteinas', 'Calcio', 'PrecioPorcion',
       'LimPorcionesDia'],
      dtype='object')

In [16]:
df

,TamPorcion,Energia,Proteinas,Calcio,PrecioPorcion,LimPorcionesDia
Alimento,,,,,,
Avena,28 g,110,4,2,30,4
Pollo,100 g,205,32,12,240,3
Huevos,2 grandes,160,13,54,130,2
LecheEntera,237cc,160,8,285,90,8
Pasta,170 g,420,4,22,200,2
Papas,260g,260,14,80,60,2


- Se desea proponer una dieta que contenga al menos 2.000 (Kcal) , 

- al menos 55 gramos de proteína 

- Y 800 (mg) de calcio. 

- Adicionalmente para garantizar cierta variedad en la dieta se establece límites de porciones por día en los alimentos.

- Con esta información se requiere encontrar la dieta que tenga el menor costo asociado y permita satisfacer los requerimientos anteriores.

In [22]:
df.PrecioPorcion[1]

240

In [26]:
len(df.LimPorcionesDia)

6

In [20]:
alimentos = df.index.tolist()

In [38]:
model = ConcreteModel(name="Alimentos Diarios")

model.alimentos_vars = Var(alimentos, bounds=(0,None), doc="Cantidad de cada alimento")

model.obj = Objective(expr=sum(df.PrecioPorcion[i]*model.alimentos_vars[i] for i in alimentos),sense=minimize, doc="El costo de la dieta")



In [39]:

model.c0 = Constraint(expr=sum(df.Energia[i]*model.alimentos_vars[i] for i in alimentos) >= 2000, doc="MinEnergia")
model.c1 = Constraint(expr=sum(df.Proteinas[i]*model.alimentos_vars[i] for i in alimentos) >= 55, doc="MinProteina")
model.c2 = Constraint(expr=sum(df.Calcio[i]*model.alimentos_vars[i] for i in alimentos) >= 800, doc="MinProteina")

## En lugar de poner todas estas restricciones hacemos un generador
#model.c3 = Constraint(expr= model.alimentos_vars[0]  <= df.LimPorcionesDia[0], doc="LimPorcionAvena")
#model.c4 = Constraint(expr= model.alimentos_vars[1]  <= df.LimPorcionesDia[0], doc="LimPorcionAvena")
#model.c5 = Constraint(expr= model.alimentos_vars[2]  <= df.LimPorcionesDia[0], doc="LimPorcionAvena")
#model.c6 = Constraint(expr= model.alimentos_vars[3]  <= df.LimPorcionesDia[0], doc="LimPorcionAvena")
#model.c7 = Constraint(expr= model.alimentos_vars[4]  <= df.LimPorcionesDia[0], doc="LimPorcionAvena")


In [51]:
#model.alimentos_vars.get_values()

In [36]:
#df.LimPorcionesDia[0],alimentos

(4, ['Avena', 'Pollo', 'Huevos', 'LecheEntera', 'Pasta', 'Papas'])

In [40]:
def con_rule(model, ind):
    return model.alimentos_vars[ind]  <= df.LimPorcionesDia[ind]

#indx = list(range(len(df.LimPorcionesDia)))
model.con = Constraint(alimentos, rule=con_rule)

In [50]:
solver = SolverFactory('ipopt')
status = solver.solve(model)

print("Status = %s" % status.solver.termination_condition)

for i in alimentos:
    print("%s = %f" % (model.alimentos_vars[i], value(model.alimentos_vars[i])))

print("Objective = %f" % value(model.obj))

Status = optimal
alimentos_vars[Avena] = 4.000000
alimentos_vars[Pollo] = 0.000000
alimentos_vars[Huevos] = 0.000000
alimentos_vars[LecheEntera] = 2.087795
alimentos_vars[Pasta] = 1.680840
alimentos_vars[Papas] = 2.000000
Objective = 764.069537
